In [1]:
import pandas as pd

In [2]:
characters = pd.read_csv("../datasets/characters.csv")

We will start by taking an overview of the data:

In [3]:
characters.describe()

,name,birth,nationality,gender,height,weight,hair_color,eye_color,build,race,...,last_appeared_chapter,affiliation,title,rank,status,death,occupation,ajah,clan,sept
count,2285,225,2146,2285,334,30,651,373,579,40,...,1572,1235,408,857,2267,386,985,360,244,200
unique,2278,142,58,3,48,15,279,120,106,1,...,60,103,118,95,17,70,123,8,17,36
top,Huan,Age of Legends,Unknown nationality,Female,Tall,Slender,Dark,Dark,Plump,Ogier,...,Prologue,White Tower,Lord,Aes Sedai,Alive,1000 NE,Soldier,Green Ajah,Unknown clan,Unknown sept
freq,2,13,479,1185,174,7,70,70,66,40,...,125,317,102,362,1498,209,241,62,83,140


In [4]:
print(f'There are {len(characters.index)} characters.')

There are 2289 characters.


As of today (22/04/2020) The **Wheel of Time wiki** has the following character counts **1189** [Female](https://wot.fandom.com/wiki/Category:Women), **1101** [Male](https://wot.fandom.com/wiki/Category:Men) and no [Unknown Gender](https://wot.fandom.com/wiki/Category:Unknown_gender) characters. This would sum to a total of **2290** characters, but the the wiki actually is counting [Wolfbrothers](https://wot.fandom.com/wiki/Category:Wolfbrothers)' Category as a Male character so the total number of **2289** is the correct one, at least up to this date.

### Dropping Characters that Don't Have Bio Details on the Wiki

First we are going to look for characters that don't have Bio details in the wiki. Looking at the order of scraped characters I could find two of those characters in the wiki. 

[Culan Cuhan](https://wot.fandom.com/wiki/Culan_Cuhan)

[Anla the Wise Counselor](https://wot.fandom.com/wiki/Anla_the_Wise_Counselor)

But, as Scrapy does a lot of paralell requests, I couldn't spot the others.

In [5]:
characters[characters["name"].isna()]

,name,birth,nationality,gender,height,weight,hair_color,eye_color,build,race,...,last_appeared_chapter,affiliation,title,rank,status,death,occupation,ajah,clan,sept
279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As can be seen, there are four characters that don't have Bio in the wiki. Let's clean these rows.

In [6]:
indexes_drop = characters[characters["name"].isna()].index
characters.drop(indexes_drop, inplace=True)

In [7]:
print(f'We are left with {len(characters.index)} characters.')

We are left with 2285 characters.


### Dealing with Nationality Data

Some characters don't have the **nationality** filled in their Bios. I am going to fill them with the value **Unkown nationality** as this is already used  in the wiki when it is explicitly stated that the nationality is unkown for a character.

In [8]:
characters["nationality"].fillna("Unknown nationality", inplace=True)

Let's look at the count for each nationality:

In [9]:
characters["nationality"].value_counts()

Unknown nationality    618
Andoran                385
Aiel                   256
Seanchan               119
Cairhienin             116
Tairen                  88
Domani                  86
Altaran                 71
Saldaean                60
Kandori                 55
Murandian               50
Taraboner               45
Shienaran               37
Amadician               36
Illianer                36
Ghealdanin              34
Arafellin               33
Far Madding             21
Borderlander            15
Mayener                 14
Malkieri                10
Malkier                  9
Saldaea                  7
Manetherenite            6
Andor                    6
Sharan                   5
Tar Valon                5
Kandor                   5
Shandalle                5
Cairhien                 4
Illian                   3
Black Hills              3
Aldesharin               3
Arad Doman               3
Murandy                  3
Shienar                  2
Sea Folk                 2
T

Looking at the data we can see that there are some values that have different names but that actually correspond to the same nationality, such as **Andor** and **Andoran**. To avoid this duplication we will map them to get a uniform name for each nationality.

In [10]:
nationality_mapping = {
    'Unknown nationality':'Unknown nationality',
    'Andoran':'Andoran',
    'Aiel':'Aiel',
    'Seanchan':'Seanchan',
    'Cairhienin':'Cairhienin',
    'Tairen':'Tairen',
    'Domani':'Domani',
    'Altaran':'Altaran',
    'Saldaean':'Saldaean',
    'Kandori':'Kandori', 
    'Murandian':'Murandian',
    'Taraboner':'Taraboner',
    'Shienaran':'Shienaran',
    'Amadician':'Amadician',
    'Illianer':'Illianer',
    'Ghealdanin':'Ghealdanin',
    'Arafellin':'Arafellin',
    'Far Madding':'Far Madding',
    'Borderlander':'Borderlander',
    'Mayener':'Mayener',
    'Malkieri':'Malkieri',
    'Malkier':'Malkieri',
    'Saldaea':'Saldaean',
    'Andor': 'Andoran',
    'Manetherenite':'Manetherenite',
    'Tar Valon':'Tar Valon',
    'Shandalle':'Shandalle',
    'Kandor':'Kandori',
    'Sharan':'Sharan',
    'Cairhien':'Cairhienin',
    'Illian':'Illianer',
    'Murandy':'Murandian',
    'Aldesharin':'Aldesharin',
    'Arad Doman':'Domani',
    'Black Hills':'Black Hills',
    'Tarabon':'Taraboner',
    'Sea Folk':'Sea Folk',
    'Shienar':'Shienaran',
    'Arafel':'Arafellin',
    'Aramaellen':'Aramaellen',
    'Aridholin':'Aridholin',
    'Shiotan':'Shiotan',
    'Altara':'Altaran',
    'Mayene':'Mayener',
    'Ghealdan':'Ghealdanin',
    'Darmovanin':'Darmovanin',
    'Dal Calainin':'Dal Calainin',
    'Dorlan':'Dorlan',                  
    'Amadicia':'Amadician',
    'Essam':'Essam',
    'Jaramide':'Jaramide',
    'Aramaelle':'Aramaelle',
    'Amayarin':'Amayarin',
    'Talmouri':'Talmouri', 
    'Masenasharin':'Masenasharin',
    'Essenian':'Essenian',
    'Hol Cuchone':'Hol Cuchone',
    'Saferin':'Saferin'
}

characters['nationality'] = characters['nationality'].map(nationality_mapping)

In [11]:
characters["nationality"].value_counts()

Unknown nationality    618
Andoran                391
Aiel                   256
Cairhienin             120
Seanchan               119
Domani                  89
Tairen                  88
Altaran                 73
Saldaean                67
Kandori                 60
Murandian               53
Taraboner               47
Illianer                39
Shienaran               39
Amadician               37
Ghealdanin              36
Arafellin               35
Far Madding             21
Malkieri                19
Mayener                 16
Borderlander            15
Manetherenite            6
Shandalle                5
Tar Valon                5
Sharan                   5
Aldesharin               3
Black Hills              3
Aridholin                2
Sea Folk                 2
Shiotan                  2
Aramaellen               2
Essenian                 1
Dal Calainin             1
Hol Cuchone              1
Jaramide                 1
Aramaelle                1
Masenasharin             1
S

In [12]:
print(f'Now we end up with {characters["nationality"].nunique()} unique nationalities')

Now we end up with 43 unique nationalities


### Know How Many Characters Don't have First Mentioned of First Appeared

Let's analyze the number of characters that having **first_mentioned_book** and **first_appeared_book** missing. These informations are going to be important for further analysis therefore missing them will not be accepted.

In [13]:
n_characters_without_first_mention_or_appearence = len(characters[characters['first_appeared_book'].isna() & characters['first_mentioned_book'].isna()])

In [14]:
print(f'There are {n_characters_without_first_mention_or_appearence} without neither column')

There are 31 without neither column


Something needs to be done about these characters. But we are going to take a deeper look at the books data in the next section and fix this problem, bear with me.

### Analyzing Book Mention and Appearences

Firstly we are going to see the book occurences and number of unique books for each of the three book attributes:

#### First Mentioned

In [85]:
characters['first_mentioned_book'].value_counts()

LOC                            118
TSR                             97
TEOTW                           77
TOM                             69
TFOH                            60
NS                              53
ACOS                            51
TWORJTWOT                       48
KOD                             41
TGS                             38
COT                             38
TGH                             38
TDR                             36
WH                              35
TPOD                            24
RPG                             21
AMOL                            17
The Wheel of Time Companion     14
Wheel of Time Companion          6
TSASG                            2
Name: first_mentioned_book, dtype: int64

In [87]:
characters['first_mentioned_book'].nunique()

20

Things are looking good. There are 20 entries. The 14 books in the series, New Spring and 5 extras that actually corresponde to 4 books 

* **TWORJTWOT** (The World of Robert Jordan's The Wheel of Time)
* **The Wheel of Time Companion** & **Wheel of Time Companion** (Both corresponde to the same book)
* **RPG** (The Wheel of Time Roleplaying Game)
* **TSASG** (The Strike at Shayol Ghul)


#### First Appeared

In [89]:
characters['first_appeared_book'].value_counts()

LOC          200
KOD          145
TSR          133
ACOS         131
TPOD         126
TOM          122
TFOH         117
NS           104
TEOTW        103
TGS           89
WH            85
TGH           79
COT           76
AMOL          61
TDR           53
CCG            3
TWORJTWOT      2
Name: first_appeared_book, dtype: int64

In [90]:
characters['first_appeared_book'].nunique()

17

Again, no problem. 17 entries. Again, 14 books in the series, New Spring and 2 extras that actually corresponde to 2 books: 

* **TWORJTWOT** (The World of Robert Jordan's The Wheel of Time)
* **CCG** (The Wheel of Time Collectible Card Game)


#### Last Appeared

In [91]:
characters['last_appeared_book'].value_counts()

AMOL         255
KOD          222
TOM          209
TGS          137
LOC          131
TSR           99
TPOD          91
COT           76
WH            67
NS            64
TFOH          53
ACOS          51
TDR           42
TGH           38
TEOTW         37
CCG            3
TWORJTWOT      2
Name: last_appeared_book, dtype: int64

In [93]:
characters['last_appeared_book'].nunique()

17

Yet again 17 entries. The same 14 books in the series, New Spring and 2 extras that actually corresponde to 2 books: 

* **TWORJTWOT** (The World of Robert Jordan's The Wheel of Time)
* **CCG** (The Wheel of Time Collectible Card Game)

#### Cleaning Characters Only Mentioned in Peripherical Material

I am going to clean the characters that don't have at least one of the three attributes related to one of the 15 books in the series (including New Spring). Of course this will also eliminate any characters that don't have any of the three columns informed in their Bio (Those 31 that we found earlier included).

In [15]:
series_books = ['NS', 'TEOTW', 'TGH', 'TDR', 'TSR','TFOH', 'LOC', 'ACOS','TPOD','WH', 'COT', 'KOD', 'TGS','TOM', 'AMOL']

In [16]:
condition = characters['first_mentioned_book'].isin(series_books) | characters['first_appeared_book'].isin(series_books) | characters['last_appeared_book'].isin(series_books)  
characters = characters[condition]

In [18]:
print(f'We remain with {len(characters.index)} characters')

We remain with 2162 characters


In [19]:
characters.to_csv('../datasets/characters_cleaned.csv')

After the cleaning we remain with **2162** characters. For analysis I intend to do in the future this cleaning is sufficient. If, by some reason, I need some other column to be cleaned I will get to it in a later post. 

That's it for now folks! Hope you enjoyed this tour through the Wheel of Time data. Feel free to contact me by any reason.

Cheers!